## Pre-processing for AT WS Model

Preprocessing steps for modelling script. In this script the following is done:
1. we combine all feature dataset (stack all feature datasets - spectral and elevation based)
2. perform mask on feature dataset stack
3. perform statistics (zs) for training data to be used for the modelling.

Once these datasets are saved, we then import these into the final modelling script (without need for reptition) as these are process intensive.

In [1]:
# install libraries
import os
import rasterio as rio
import numpy as np
import pandas as pd

import geopandas as gpd
from osgeo import osr
from osgeo import gdal, gdal_array

from rsgislib import zonalstats

In [2]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
# create directories to retrive and store raster data
# feature datasets (stack layer)
fn = 'C:/Data/atulip/ws/subset_area/site0/all/feature_dataset/' # file with individual feature datasets
out_fn = 'C:/Data/atulip/ws/subset_area/site0/all/pre/' # file to export stacked dataset, masked dataset
stack = out_fn + 'stack.tif'

# mask datasets
mask_fn = 'C:/Data/atulip/ws/subset_area/site0/site0_tree_mask_final.tif'
mask_array_fn = out_fn + 'masked_stack.tif'

In [4]:
# get layer name 
file_ls = [f for f in os.listdir(fn) if os.path.isfile(os.path.join(fn, f))]

# generate filepaths for each file in the feature dataset
ls = []
for i in range(0,len(file_ls)):
    id = os.path.join(fn+file_ls[i])
    # print(id)
    ls.append(id)

#### 1. Create raster stack (45 feature datasets)

In [5]:
# load in the first feature dataset and read in metadata
with rio.open(ls[1]) as src0:
    meta = src0.meta
    ds_1 = src0.read()
    ds_1 = ds_1[0,:,:] # to get two dimensions of array and utilise later when saving/exporting result
# update to the total no of layers
meta.update(count = len(ls), dtype = 'float32', nodata = -9999) # update metadata band count, datatype and nodata as -999 instead of 0

In [6]:
# stack raster from list and write - RUN ONCE, unless new covariates/layers are added in - to save time and computing space
with rio.open(stack, 'w', **meta) as dst:
    for id, layer in enumerate(ls, start = 1):
        with rio.open(layer) as src1:
            dst.write_band(id, src1.read(1))
    dst.descriptions = tuple(file_ls)

In [7]:
# load in stacked raster and prepare for step 2
# open new stack (or re-open previously crated stack), this must also then be reshaped
with rio.open(stack) as src:
    # get metadata from dataset
    bcount = src.count
    tags = src.tags()
    stack_r = src.read()
    src_meta = src.profile

In [8]:
# rearrange shape of the image stack to train and predict on
img = np.transpose(stack_r,(1,2,0))
img.shape # must be col, row, band

(9423, 22127, 43)

#### 2. Create masked stack feature dataset

In [9]:
# load in a mask image to make the image for prediction - skip if masked previously
with rio.open(mask_fn) as dt:
    raster_array = dt.read()
    bc = dt.count

In [10]:
# change values (currently 0 = non-tree(mask), 1=tree (no mask))
# Replace 0 (non-tree - to mask) with True and 1 (tree - no mask) with False
mask_array = np.select([raster_array == -9999, raster_array == 1], [1,0], raster_array)
# change dimension of array
mask_array = np.transpose(mask_array,(1,2,0))

In [11]:
# repeat array across axis 1, x number of bands in the stacked array
mask = np.repeat(mask_array, bcount, axis = 2)
mask.shape # shape must be row, col, band

(9423, 22127, 43)

In [12]:
# perform mask on the image
X_masked_array = np.ma.masked_array(img, mask=mask, fill_value = -9999)
# X_masked_array

In [13]:
# rearrange shape of masked stack to write stack to local folder (bands, rows, cols)
masked_array = np.transpose(X_masked_array,(2,0,1))
# masked_array# rearrange shape of masked stack to write stack to local folder (bands, rows, cols)

In [14]:
# write masked array stack to file - skip if already created, mask stack is opened later when refining the feature selection (and reshaped)
with rio.open(mask_array_fn, 'w', **meta) as dst:
    dst.write(masked_array)
    dst.descriptions = tuple(file_ls)

In [15]:
# reshape numpy to match the image
X_array = X_masked_array[:,:,:bcount].reshape(-1, X_masked_array.shape[-1])
print(X_array.shape) # is a numpy ndarray

(208502721, 43)


#### 3. Calculate zonal statistics on ground training data points using masked stack

##### Create proportional training data

In [16]:
# FIRST - WHERE TRAINING DATA CLASSES ARE NOT PROPORTIONAL, perform a random sampling to ensure these are proportional
# open current training data
td_fn = 'C:/Data/atulip/ws/subset_area/site0/training_data/'
td_layer = td_fn + 'site0_training.shp'
# open training data using gdal
input_data = gpd.read_file(td_layer)
input_data.head()

,Feature,class_id,geometry
0,Other,2.0,POINT (417624.429 8468073.953)
1,African Tulip,1.0,POINT (417648.292 8468031.229)
2,Other,2.0,POINT (417535.969 8468005.374)
3,Other,2.0,POINT (418181.133 8469393.981)
4,Other,2.0,POINT (417828.325 8470171.153)


In [17]:
# SPLIT BASED ON CLASSES
at = input_data[input_data['class_id'] == 1.0]
other = input_data[input_data['class_id'] == 2.0]

# randomly select the same nunmber of samples of class AT for OTHER class
other_sample = other.sample(n=349)

# merge the other samples and at to create the training dataset
t_data = pd.concat([at, other_sample])
t_data # created 668 rows of total training_data

,Feature,class_id,geometry
1,African Tulip,1.0,POINT (417648.292 8468031.229)
5,African Tulip,1.0,POINT (417808.243 8467342.295)
7,African Tulip,1.0,POINT (417997.783 8467241.584)
8,African Tulip,1.0,POINT (417728.569 8467852.090)
10,African Tulip,1.0,POINT (417989.324 8467659.639)
...,...,...,...
756,Other,2.0,POINT (416353.615 8467790.857)
655,Other,2.0,POINT (417633.405 8469922.890)
927,Other,2.0,POINT (421852.466 8469285.770)
437,Other,2.0,POINT (415855.188 8467189.524)


In [18]:
# get breakdown of class and total class count = must be equal to each other
t_data['class_id'].value_counts()

class_id
1.0    349
2.0    349
Name: count, dtype: int64

In [19]:
# EXPORT
lyr = 'gcp_prop.gpkg'
t_data.to_file(td_fn + lyr)

##### Calculate zonal statistics on ground training data (point value)

In [20]:
# file data file
vec_file = 'C:/Data/atulip/ws/subset_area/site0/training_data/'
lyr = 'gcp_prop.gpkg'

In [21]:
# extract pixel value
for i in range(bcount):
    print('calculating.....', file_ls[i][:-4])
    zonalstats.ext_point_band_values_file(
        vec_file + lyr,
        lyr[:-5],
        out_fn + 'masked_stack.tif',
        img_band = i+1,
        min_thres = -1000,
        max_thres = 70000, # double check these values
        out_no_data_val = -9999,
        out_field = file_ls[i][:-4]
    )

calculating..... ASM


100%|████████████████████████████████████████████████████████████████████████████| 698/698 [00:05<00:00, 136.81it/s]


calculating..... blue


100%|████████████████████████████████████████████████████████████████████████████| 698/698 [00:03<00:00, 186.32it/s]


calculating..... blue_laplacian


...

100%|████████████████████████████████████████████████████████████████████████████| 698/698 [00:04<00:00, 145.71it/s]


calculating..... r_var


100%|████████████████████████████████████████████████████████████████████████████| 698/698 [00:04<00:00, 162.75it/s]


calculating..... var


100%|████████████████████████████████████████████████████████████████████████████| 698/698 [00:04<00:00, 149.14it/s]


##### Calculate zonal statistics on ground training data (polygon value)

###### Calculating mean values based on segmentation. Polygons selected by point in polygon.

In [22]:
# select training polygons from segmentation layer using points gcp training data
# locate training data
wd = 'C:/Data/atulip/ws/subset_area/site0/'
gcp = wd + 'training_data/gcp_prop.gpkg'
# locate segment vector layer
seg_vec = wd + 'all/segmentation/ps_all_stack_seg_mp20.gpkg'

In [23]:
# load in training data 
training_data = gpd.read_file(gcp)
seg_data = gpd.read_file(seg_vec)

In [24]:
# peform spatial join to select training polygons
poly_class = gpd.sjoin(seg_data, training_data)
poly_class.head()

,PXLVAL,geometry,index_right,Feature,class_id,asm,blue,blue_laplacian,blue_maxd,b_var,...,nir,nir_laplacian,nir_maxd,n_var,red,redgreen,red_laplacian,red_maxd,r_var,var
11404,13059,"POLYGON ((418044.000 8470562.000, 418044.000 8...",234,African Tulip,1.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.000000,-9999.0,-9999.0,-9999.0,-9999.0
12667,10777,"POLYGON ((411430.000 8470566.000, 411430.000 8...",349,Other,2.0,1.0,322.0,435.0,498.0,0.0,...,4666.0,61374.0,4451.0,0.0,549.0,-0.170068,4320.0,4674.0,0.0,0.0
16582,16492,"POLYGON ((417317.000 8470555.500, 417317.000 8...",41,African Tulip,1.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.000000,-9999.0,-9999.0,-9999.0,-9999.0
29279,8106,"POLYGON ((411911.500 8470571.000, 411911.500 8...",602,Other,2.0,1.0,400.0,400.0,622.0,0.0,...,2944.0,63616.0,2401.0,0.0,975.0,0.126516,1737.0,2705.0,0.0,0.0
34211,32978,"POLYGON ((411446.000 8470525.000, 411446.000 8...",427,Other,2.0,1.0,252.0,233.0,413.0,0.0,...,4684.0,61262.0,4655.0,0.0,473.0,-0.061508,4241.0,4849.0,0.0,0.0


In [25]:
pp = poly_class[['class_id', 'geometry']]
pp.head()

,class_id,geometry
11404,1.0,"POLYGON ((418044.000 8470562.000, 418044.000 8..."
12667,2.0,"POLYGON ((411430.000 8470566.000, 411430.000 8..."
16582,1.0,"POLYGON ((417317.000 8470555.500, 417317.000 8..."
29279,2.0,"POLYGON ((411911.500 8470571.000, 411911.500 8..."
34211,2.0,"POLYGON ((411446.000 8470525.000, 411446.000 8..."


In [26]:
# export polygon layer attributed with training classes - becomes training polygons
pp.to_file(wd + 'training_data/training_poly.gpkg')

###### Calculate zonal stats using polgyon

In [27]:
vec_file = wd + 'training_data/'
lyr = 'training_poly.gpkg'
# bcount = 45

In [28]:
for i in range(bcount):
    print('calculating.....', file_ls[i][:-4])
    zonalstats.calc_zonal_band_stats_file(
        vec_file = vec_file + lyr,
        vec_lyr = lyr[:-5],
        input_img = out_fn + 'masked_stack.tif',
        img_band = i+1,
        min_thres = -1000,
        max_thres = 70000,
        out_no_data_val = -9999,
        mean_field = file_ls[i][:-4]
)

calculating..... ASM


243951it [00:32, 7527.34it/s]                                                                                       


calculating..... blue


243951it [00:32, 7563.16it/s]                                                                                       


calculating..... blue_laplacian


...

243951it [00:34, 7021.50it/s]                                                                                       


calculating..... r_var


243951it [00:31, 7624.49it/s]                                                                                       


calculating..... var


243951it [00:36, 6641.21it/s]                                                                                       


In [29]:
# open vector dataset
# read in training/testing data - with zonal statistics/pixel values
input_data = gpd.read_file(os.path.join(vec_file + lyr))
input_data = input_data.dropna() # drop na values
input_data.head() # preview shapefile

,class_id,asm,blue,blue_laplacian,blue_maxd,b_var,contrast,correlation,dem,dem_lap,...,nir_laplacian,nir_maxd,n_var,red,redgreen,red_laplacian,red_maxd,r_var,var,geometry
0,1.0,1.0,251.000000,317.333344,490.000000,0.000000,0.0,1.0,13.563141,1.820106,...,62143.332031,4094.666748,4.666667,553.000000,-0.004821,3161.333252,4267.666504,0.000000,0.0,"POLYGON ((418044.000 8470562.000, 418044.000 8..."
1,2.0,1.0,326.885101,432.353180,531.846802,0.000000,0.0,1.0,94.899628,-0.141895,...,61271.457031,4510.659668,0.600000,570.872314,-0.137265,4394.221191,4757.425293,0.000000,0.0,"POLYGON ((411430.000 8470566.000, 411430.000 8..."
2,1.0,1.0,271.444458,407.888885,793.777771,0.111111,0.0,1.0,3.999743,-0.546680,...,62285.000000,4352.222168,5.888889,494.777771,-0.097107,3363.666748,4496.666504,0.222222,0.0,"POLYGON ((417317.000 8470555.500, 417317.000 8..."
3,2.0,1.0,435.585571,309.983490,639.480469,0.000000,0.0,1.0,80.613716,-0.006478,...,63573.472656,2486.376953,0.202703,962.057068,0.123762,1752.559326,2796.502197,0.000000,0.0,"POLYGON ((411911.500 8470571.000, 411911.500 8..."
4,2.0,1.0,225.142853,261.111115,452.317474,0.000000,0.0,1.0,99.454094,-2.103567,...,61611.808594,4637.507812,4.285714,418.269836,-0.083401,3902.444336,4783.349121,0.000000,0.0,"POLYGON ((411446.000 8470525.000, 411446.000 8..."


In [30]:
# write vec layer to folder
input_data.to_file(vec_file + 'gcp_poly_mean_val.gpkg')